<a href="https://colab.research.google.com/github/biruk50/Medium_articles/blob/main/testing_set_covering_algorithms_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project requires Python 3.7 or above:



In [ ]:
import sys

assert sys.version_info >= (3, 7)

In [ ]:
!pip install pulp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 55.8 MB/s eta 0:00:00


In [ ]:
import timeit
import heapq
import pulp


In [88]:
#this generates an examples set where not all cameras might cover the stadiums_needed
import random

# Number of stadium areas
num_stadiums = 100

# Create the set of all unique stadium areas
stadiums_needed = set(range(1, num_stadiums + 1))

# Number of cameras
num_cameras = 30

# Define a function to generate random camera coverage
def generate_random_camera_coverage(stadiums, min_coverage=5, max_coverage=15):
    """Generate a random set of stadium areas covered by a camera."""
    stadiums_list = list(stadiums)  # Convert the set to a list
    coverage_size = random.randint(min_coverage, max_coverage)
    return set(random.sample(stadiums_list, coverage_size))  # Sample from the list

# Create a dictionary of camera locations and their randomly assigned coverage areas
camera_locations = {}
for i in range(1, num_cameras + 1):
    camera_name = f"camera{i}"
    camera_locations[camera_name] = generate_random_camera_coverage(stadiums_needed)

# Print the generated test case
print("stadiums_needed =", stadiums_needed)
print("camera_locations = {")
for camera, coverage in camera_locations.items():
    print(f'    "{camera}": {coverage},')
print("}")


stadiums_needed = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}
camera_locations = {
    "camera1": {67, 35, 76, 12, 81, 19, 84, 20, 87, 88, 62, 26, 29, 94, 95},
    "camera2": {1, 68, 38, 7, 8, 9, 45, 14, 13, 83, 54},
    "camera3": {98, 2, 99, 68, 5, 72, 74, 12, 31, 85, 23, 60, 63},
    "camera4": {32, 96, 34, 36, 5, 69, 74, 42, 14, 80, 53, 26},
    "camera5": {33, 5, 39, 74, 12, 45, 76, 15, 80, 18, 85, 26, 93, 63},
    "camera6": {96, 98, 99, 100, 71, 9, 42, 44, 77, 78, 76, 18, 58, 30},
    "camera7": {36, 4, 6, 5, 7, 71, 10, 43, 48, 83, 51, 20, 86, 92},
    "camera8": {97, 14, 50, 88, 59, 61},
    "camera9": {100, 39, 40, 9, 76, 45, 80, 20, 

In [104]:
import random

# Number of stadium areas
num_stadiums = 20

# Create the set of all unique stadium areas
stadiums_needed = set(range(1, num_stadiums + 1))

# Number of cameras
num_cameras = 20

# Define a function to generate random camera coverage
def generate_random_camera_coverage(stadiums):
    """Generate a random set of stadium areas covered by a camera."""
    stadiums_list = list(stadiums)  # Convert the set to a list
    # Ensure coverage_size is not larger than the population
    coverage_size = random.randint(len(stadiums)//8, min(len(stadiums)//4, len(stadiums_list))) #min,max coverages
    return set(random.sample(stadiums_list, coverage_size))  # Sample from the list

# Create a dictionary of camera locations and their randomly assigned coverage areas
camera_locations = {}
for i in range(1, num_cameras + 1):
    camera_name = f"camera{i}"
    camera_locations[camera_name] = generate_random_camera_coverage(stadiums_needed)

# Check which stadiums are not covered by any camera
covered_stadiums = set()
for coverage in camera_locations.values():
    covered_stadiums.update(coverage)

uncovered_stadiums = stadiums_needed - covered_stadiums

# Assign uncovered stadiums to random cameras until all stadiums are covered
if uncovered_stadiums:
    camera_names = list(camera_locations.keys())
    for stadium in uncovered_stadiums:
        random_camera = random.choice(camera_names)
        camera_locations[random_camera].add(stadium)

# Print the generated test case
print("stadiums_needed =", stadiums_needed)
print("camera_locations = {")
for camera, coverage in camera_locations.items():
    print(f'    "{camera}": {coverage},')
print("}")

stadiums_needed = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
camera_locations = {
    "camera1": {16, 1, 11},
    "camera2": {11, 5, 6, 15},
    "camera3": {18, 7},
    "camera4": {9, 6},
    "camera5": {20, 5, 14, 15},
    "camera6": {19, 5},
    "camera7": {5, 6},
    "camera8": {2, 3, 5, 15, 18},
    "camera9": {18, 7},
    "camera10": {12, 6, 14, 7},
    "camera11": {2, 6, 15},
    "camera12": {17, 3, 12, 20},
    "camera13": {12, 20, 14},
    "camera14": {10, 11, 13, 14},
    "camera15": {18, 19, 14},
    "camera16": {2, 3, 11, 14, 20},
    "camera17": {4, 5, 9, 13, 16},
    "camera18": {16, 1},
    "camera19": {3, 5, 14},
    "camera20": {8, 4},
}


In [ ]:
stadiums_needed = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
camera_locations = {
        "camera1": {1, 3, 8, 11, 22, 29},
        "camera2": {20, 6, 15},
        "camera3": {2, 6, 7, 26, 30},
        "camera4": {4, 12, 13, 16, 26},
        "camera5": {10, 4, 6},
        "camera6": {8, 1, 26, 22},
        "camera7": {1, 7, 8, 21, 29},
        "camera8": {8, 16, 2, 5},
        "camera9": {9, 10, 25},
        "camera10": {6, 11, 12, 16, 21, 29},
        "camera11": {16, 4, 30},
        "camera12": {8, 4, 13},
        "camera13": {7, 10, 12, 17, 18, 19, 25},
        "camera14": {6, 7, 15, 22, 25, 26},
        "camera15": {4, 14, 20, 23, 24, 25, 30},
        "camera16": {16, 17, 20, 7},
        "camera17": {24, 1, 4, 30},
        "camera18": {1, 5, 9, 13, 20, 23},
        "camera19": {1, 9, 22, 26, 27},
        "camera20": {7, 8, 23, 24, 26},
        "camera21": {1, 4, 5, 9, 11, 18},
        "camera22": {17, 11, 5, 25},
        "camera23": {7, 19, 20, 23, 28},
        "camera24": {2, 5, 15},
        "camera25": {1, 4, 25, 23},
        "camera26": {2, 3, 15, 16, 24, 28, 29},
        "camera27": {1, 3, 16, 21, 23, 30},
        "camera28": {3, 4, 11, 14, 22, 25, 26},
        "camera29": {2, 10, 12, 22},
        "camera30": {25, 2, 20, 22},
        "camera31": {4, 7, 9, 11, 14, 19},
        "camera32": {2, 8, 10, 11, 18, 29},
        "camera33": {25, 10, 18, 14},
        "camera34": {16, 27, 29},
        "camera35": {1, 8, 16, 24, 26},
        "camera36": {6, 8, 9, 10, 13, 23, 26},
        "camera37": {3, 4, 9, 15, 25, 26},
        "camera38": {2, 3, 15, 18, 21, 28, 30},
        "camera39": {18, 19, 29},
        "camera40": {8, 9, 20, 15},
        "camera41": {6, 7, 8, 15, 17, 20},
        "camera42": {10, 26, 7},
        "camera43": {1, 10, 28},
        "camera44": {24, 2, 4, 13},
        "camera45": {8, 10, 11, 24, 28},
        "camera46": {6, 11, 14, 20, 23, 27},
        "camera47": {16, 10, 14},
        "camera48": {9, 21, 22, 27, 29},
        "camera49": {2, 5, 6, 27, 30},
        "camera50": {8, 1, 30},
        "camera51": {8, 10, 19, 23},
        "camera52": {3, 9, 13, 19, 23, 24, 26},
        "camera53": {8, 19, 5, 7},
        "camera54": {1, 4, 8, 9, 12, 29},
        "camera55": {24, 25, 16},
        "camera56": {1, 2, 3, 8, 17, 22, 27},
        "camera57": {30, 14, 22},
        "camera58": {26, 27, 28, 6},
        "camera59": {1, 10, 12, 17, 18, 26, 29},
        "camera60": {24, 8, 14},
}

#this handles all cases like all the cameras are not fully covering the the stadiums_needed

def latest_improved_set_covering(stadiums_needed, camera_locations):
    # Dictionary to store how many cameras cover each stadium and which cameras
    Count = {}
    final_camera_locations = set()
    unusable_camera_locations=set()

    max_count = 1  # Track the highest number of location covered by different cameras
    min_count = 1  # Track the lowest number of location covered by different cameras

    # Count occurrences of each stadium in the camera_locations
    for camera, stadiums in camera_locations.items():
        for stadium in stadiums:
            if stadium in Count:
                Count[stadium][0] += 1
                Count[stadium][1].add(camera)

                max_count = max(Count[stadium][0],max_count)
                min_count = min(Count[stadium][0], min_count)
            else:
                Count[stadium] = [1, set([camera])]  # Initialize with count 1 and set of cameras
                min_count = 1


    # Function to calculate the number of stadiums in common with stadiums_needed
    def stadiums_in_common(camera):
        return len(stadiums_needed & camera_locations[camera])

    # Function to add the selected camera to the final list and update stadiums_needed
    def add_to_final(camera, stadiums_needed):
        if camera in final_camera_locations:
            return
        final_camera_locations.add(camera)
        print(camera)
        stadiums_needed -= camera_locations[camera]

    min_set =set()
    i = min_count  # This keeps track of the Count values

    if i == 1:
      #this is to handle element that are covered by just 1 set
      for stadium, value in Count.items():
            if value[0] == 1:
                min_set.update(value[1])
      while min_set:
        add_to_final(min_set.pop(), stadiums_needed)

      i=2

    while stadiums_needed and i < max_count:
        # Create the maximum heap (priority queue)
        heap = []
        for stadium, value in Count.items():
            if value[0] == i or value[0] == i + 1:
                for camera in value[1]:
                    if camera not in final_camera_locations and camera not in unusable_camera_locations:
                        num_common_stadiums = stadiums_in_common(camera)
                        heapq.heappush(heap, (-num_common_stadiums, value[0], camera))  # Negate num_common_stadiums for max heap

        while heap and stadiums_needed:
            _, _, best_camera = heapq.heappop(heap)

            if stadiums_in_common(best_camera) == 0:
                # Add all remaining cameras in the heap to unusable_camera_locations
                unusable_camera_locations.update([camera for _, _, camera in heap])
                unusable_camera_locations.add(best_camera)  # Add the current best_camera as well
                heap.clear()
                break

            # Rebuild the heap after add_to_final
            heap = []
            for stadium, value in Count.items():
                if value[0] == i or value[0] == i + 1:
                    for camera in value[1]:
                        if camera not in final_camera_locations and camera not in unusable_camera_locations:
                            num_common_stadiums = stadiums_in_common(camera)
                            heapq.heappush(heap, (-num_common_stadiums, value[0], camera))

        if not stadiums_needed:
            return final_camera_locations

        i+= 1  # Increase i if no cameras with i coverage

    return final_camera_locations

def improved_set_covering(stadiums_needed, camera_locations):
    # Dictionary to store how many cameras cover each stadium and which cameras
    Count = {}
    final_camera_locations = set()
    unusable_camera_locations=set()

    max_count = 1  # Track the highest number of location covered by different cameras

    # Count occurrences of each stadium in the camera_locations
    for camera, stadiums in camera_locations.items():
        for stadium in stadiums:
            if stadium in Count:
                Count[stadium][0] += 1
                Count[stadium][1].add(camera)

                max_count = max(Count[stadium][0],max_count)
            else:
                Count[stadium] = [1, set([camera])]  # Initialize with count 1 and set of cameras

    # Sort Count items by value[0] in ascending order
    sorted_count = sorted(Count.items(), key=lambda item: item[1][0])

    # Print sorted elements
    for stadium, value in sorted_count:
        print(f"{stadium}: {value}")

    # Function to calculate the number of stadiums in common with stadiums_needed
    def stadiums_in_common(camera):
        return len(stadiums_needed & camera_locations[camera])

    # Function to add the selected camera to the final list and update stadiums_needed
    def add_to_final(camera, stadiums_needed):
        if camera in final_camera_locations:
            return
        final_camera_locations.add(camera)
        print(camera)
        stadiums_needed -= camera_locations[camera]

    i = 1  # This keeps track of the Count values
    min_set =set()
    while stadiums_needed and i <= max_count:
        for stadium, value in Count.items():
            if value[0] == i:
                min_set.update(value[1])  # Add all cameras covering the stadium with count i

        min_set-=final_camera_locations

        while min_set and stadiums_needed:
            if i >1:
              best_camera = max(min_set, key=stadiums_in_common)

              if stadiums_in_common(best_camera) == 0:
                #we first update unusable_camera_locations
                unusable_camera_locations.update(min_set)

                # If the camera does not cover any needed stadiums, clear the set and break
                min_set.clear()
                i+=1
                break
              else:
                min_set.discard(best_camera)
                add_to_final(best_camera, stadiums_needed)

            elif i == 1:  # we don't have to go down the greedy approach , since all camera location that cover a stadium no other camera covers need to be included
              add_to_final(min_set.pop(), stadiums_needed)

        if not stadiums_needed:
            return final_camera_locations

        i+= 1  # Increase i if no cameras with i coverage

    return final_camera_locations

# Greedy Algorithm
def greedy_set_covering(stadiums_needed, camera_locations):
    final_locations =[]
    while stadiums_needed:
        best_location = None
        area_covered = set()
        for location, stadium_area in camera_locations.items():
            covered = stadiums_needed & stadium_area
            if len(covered) > len(area_covered):
                best_location = location
                area_covered = covered
        if best_location:
            final_locations.append(best_location)
            stadiums_needed -= area_covered
            del camera_locations[best_location]
        else:
            break
    return final_locations

# ILP Algorithm
def ilp_set_covering(stadiums_needed, camera_locations):
    prob = pulp.LpProblem("SetCovering", pulp.LpMinimize)
    location_vars = pulp.LpVariable.dicts("Station", camera_locations.keys(), 0, 1, pulp.LpBinary)
    prob += pulp.lpSum([location_vars[location] for location in camera_locations.keys()])
    for stadium in stadiums_needed:
        prob += pulp.lpSum([location_vars[location] for location in camera_locations.keys() if stadium in camera_locations[location]]) >= 1
    prob.solve()
    selected_locations = [locations for locations in camera_locations.keys() if location_vars[locations].varValue == 1]
    return selected_locations

# Heap-Based Algorithm
def heap_set_covering(stadiums_needed, camera_locations):
    final_locations =[]
    Count = {state: 0 for state in stadiums_needed}

    # Count occurrences of each area in the camera_locations
    for location, stadium_area in camera_locations.items():
        for area in stadium_area:
            if area in Count:
                Count[area] += 1

    heap = []

    # Function to calculate the number of stadium_area in common with stadium_needed
    def area_in_common(stadium_area):
        return len(stadiums_needed & stadium_area)

    # Get the maximum count value
    max_count = max(Count.values(), default=0)

    # Iterate over count values from 1 to max_count
    for count in range(1, max_count + 1):
        for location, stadium_area in camera_locations.items():
            if any(Count[state] == count for state in stadium_area):
                heapq.heappush(heap, (count, -area_in_common(stadium_area), location, stadium_area))

    # Process the heap
    while stadiums_needed and heap:
        _, _, location, stadium_area = heapq.heappop(heap)
        if location in final_locations:
            continue
        final_locations.append(location)
        stadiums_needed -= stadium_area
        for st in stadium_area:
            if st in Count:
                del Count[st]
        del camera_locations[location]

        # Rebuild the heap with remaining areas
        heap = []
        for count in range(1, max_count + 1):
            for location, stadium_area in camera_locations.items():
                if any(area in Count and Count[area] == count for area in stadium_area):
                    heapq.heappush(heap, (count, -area_in_common(stadium_area), location, stadium_area))

    return final_locations

# Measure runtime
def measure_runtime(func, *args):
    start_time = timeit.default_timer()
    result = func(*args)
    end_time = timeit.default_timer()
    return result, end_time - start_time

# Run and measure algorithms
latest_improved_set_covering_result, latest_improved_time = measure_runtime(latest_improved_set_covering, stadiums_needed.copy(), camera_locations.copy())
improved_result, improved_time = measure_runtime(improved_set_covering, stadiums_needed.copy(), camera_locations.copy())
greedy_result, greedy_time = measure_runtime(greedy_set_covering, stadiums_needed.copy(), camera_locations.copy())
heap_result, heap_time = measure_runtime(heap_set_covering, stadiums_needed.copy(), camera_locations.copy())
ilp_result, ilp_time = measure_runtime(ilp_set_covering, stadiums_needed.copy(), camera_locations.copy())

# Print results
print(f"Latest Improved Algorithm: length: {len(latest_improved_set_covering_result)} , {latest_improved_set_covering_result}, Time: {latest_improved_time}")
print(f"Improved Algorithm: length: {len(improved_result)} , {improved_result}, Time: {improved_time}")
print(f"Greedy Algorithm: length: {len(greedy_result)} ,{greedy_result}, Time: {greedy_time}")
print(f"Heap-Based Algorithm: {len(heap_result)}, Time: {heap_time}")
print(f"ILP Algorithm: length: {len(ilp_result)} ,{ilp_result}, Time: {ilp_time}")

# Summary
print(f"Improved Algorithm Time: {improved_time}")
print(f"Greedy Algorithm Time: {greedy_time}")
print(f"Heap-Based Algorithm Time: {heap_time}")
print(f"ILP Algorithm Time: {ilp_time}")


In [116]:

# ILP result cameras
ilp_cameras = ['camera13', 'camera26', 'camera18', 'camera28', 'camera49', 'camera7']

# Combine the stadiums covered by the selected ILP cameras
covered_stadiums = set()
for camera in ilp_cameras:
    covered_stadiums.update(camera_locations[camera])

# Check if all stadiums are covered
all_stadiums_covered = stadiums_needed.issubset(covered_stadiums)

# Print the result
if all_stadiums_covered:
    print("All stadiums are covered by the selected ILP cameras.")
else:
    missing_stadiums = stadiums_needed - covered_stadiums
    print("Not all stadiums are covered. Missing stadiums:", missing_stadiums)


All stadiums are covered by the selected ILP cameras.


In [ ]:
states_needed = set(["mt", "wa", "or", "id", "nv", "ut", "ca", "az"])

stations = {
    "kone": set(["id", "nv", "ut"]),
    "ktwo": set(["wa", "id", "mt"]),
    "kthree": set(["or", "nv", "ca"]),
    "kfour":  set(["nv", "ut"]),
    "kfive": set(["ca", "az"])
}
# Greedy Algorithm
def greedy_set_covering(states_needed, stations):
    final_stations = set()
    while states_needed:
        best_station = None
        states_covered = set()
        for station, station_states in stations.items():
            covered = states_needed & station_states
            if len(covered) > len(states_covered):
                best_station = station
                states_covered = covered
        if best_station:
            final_stations.add(best_station)
            states_needed -= states_covered
            del stations[best_station]
        else:
            break
    return final_stations

# ILP Algorithm
def ilp_set_covering(states_needed, stations):
    prob = pulp.LpProblem("SetCovering", pulp.LpMinimize)
    station_vars = pulp.LpVariable.dicts("Station", stations.keys(), 0, 1, pulp.LpBinary)
    prob += pulp.lpSum([station_vars[station] for station in stations.keys()])
    for state in states_needed:
        prob += pulp.lpSum([station_vars[station] for station in stations.keys() if state in stations[station]]) >= 1
    prob.solve()
    selected_stations = [station for station in stations.keys() if station_vars[station].varValue == 1]
    return selected_stations

# Heap-Based Algorithm
def heap_set_covering(states_needed, stations):
    final_stations = set()
    Count = {state: 0 for state in states_needed}

    # Count occurrences of each state in the stations
    for station, station_states in stations.items():
        for state in station_states:
            if state in Count:
                Count[state] += 1

    heap = []

    # Function to calculate the number of states in common with states_needed
    def states_in_common(station_states):
        return len(states_needed & station_states)

    # Get the maximum count value
    max_count = max(Count.values(), default=0)

    # Iterate over count values from 1 to max_count
    for count in range(1, max_count + 1):
        for station, station_states in stations.items():
            if any(Count[state] == count for state in station_states):
                heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    # Process the heap
    while states_needed and heap:
        _, _, station, station_states = heapq.heappop(heap)
        if station in final_stations:
            continue
        final_stations.add(station)
        states_needed -= station_states
        for st in station_states:
            if st in Count:
                del Count[st]
        del stations[station]

        # Rebuild the heap with remaining stations
        heap = []
        for count in range(1, max_count + 1):
            for station, station_states in stations.items():
                if any(state in Count and Count[state] == count for state in station_states):
                    heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    return final_stations

# Measure runtime
def measure_runtime(func, *args):
    start_time = timeit.default_timer()
    result = func(*args)
    end_time = timeit.default_timer()
    return result, end_time - start_time

# Run and measure algorithms
greedy_result, greedy_time = measure_runtime(greedy_set_covering, states_needed.copy(), stations.copy())
heap_result, heap_time = measure_runtime(heap_set_covering, states_needed.copy(), stations.copy())
ilp_result, ilp_time = measure_runtime(ilp_set_covering, states_needed.copy(), stations.copy())

# Print results
print(f"Greedy Algorithm: {greedy_result}, Time: {greedy_time}")
print(f"Heap-Based Algorithm: {heap_result}, Time: {heap_time}")
print(f"ILP Algorithm: {ilp_result}, Time: {ilp_time}")

# Summary
print(f"Greedy Algorithm Time: {greedy_time}")
print(f"Heap-Based Algorithm Time: {heap_time}")
print(f"ILP Algorithm Time: {ilp_time}")

Greedy Algorithm: {'kfive', 'kthree', 'kone', 'ktwo'}, Time: 1.4332999995758655e-05
Heap-Based Algorithm: {'kfive', 'ktwo', 'kfour', 'kthree'}, Time: 7.308799999350413e-05
ILP Algorithm: ['ktwo', 'kthree', 'kfour', 'kfive'], Time: 0.011743835999993735
Greedy Algorithm Time: 1.4332999995758655e-05
Heap-Based Algorithm Time: 7.308799999350413e-05
ILP Algorithm Time: 0.011743835999993735


In [ ]:
states_needed = {"s1", "s2", "s3", "s4", "s5", "s6"}

stations = {
    "A": {"s1", "s2", "s3"},
    "B": {"s1", "s4"},
    "C": {"s2", "s5"},
    "D": {"s3", "s6"},
    "E": {"s4", "s5", "s6"}
}

# Greedy Algorithm
def greedy_set_covering(states_needed, stations):
    final_stations = set()
    while states_needed:
        best_station = None
        states_covered = set()
        for station, station_states in stations.items():
            covered = states_needed & station_states
            if len(covered) > len(states_covered):
                best_station = station
                states_covered = covered
        if best_station:
            final_stations.add(best_station)
            states_needed -= states_covered
            del stations[best_station]
        else:
            break
    return final_stations

# ILP Algorithm
def ilp_set_covering(states_needed, stations):
    prob = pulp.LpProblem("SetCovering", pulp.LpMinimize)
    station_vars = pulp.LpVariable.dicts("Station", stations.keys(), 0, 1, pulp.LpBinary)
    prob += pulp.lpSum([station_vars[station] for station in stations.keys()])
    for state in states_needed:
        prob += pulp.lpSum([station_vars[station] for station in stations.keys() if state in stations[station]]) >= 1
    prob.solve()
    selected_stations = [station for station in stations.keys() if station_vars[station].varValue == 1]
    return selected_stations

# Heap-Based Algorithm
def heap_set_covering(states_needed, stations):
    final_stations = set()
    Count = {state: 0 for state in states_needed}

    # Count occurrences of each state in the stations
    for station, station_states in stations.items():
        for state in station_states:
            if state in Count:
                Count[state] += 1

    heap = []

    # Function to calculate the number of states in common with states_needed
    def states_in_common(station_states):
        return len(states_needed & station_states)

    # Get the maximum count value
    max_count = max(Count.values(), default=0)

    # Iterate over count values from 1 to max_count
    for count in range(1, max_count + 1):
        for station, station_states in stations.items():
            if any(Count[state] == count for state in station_states):
                heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    # Process the heap
    while states_needed and heap:
        _, _, station, station_states = heapq.heappop(heap)
        if station in final_stations:
            continue
        final_stations.add(station)
        states_needed -= station_states
        for st in station_states:
            if st in Count:
                del Count[st]
        del stations[station]

        # Rebuild the heap with remaining stations
        heap = []
        for count in range(1, max_count + 1):
            for station, station_states in stations.items():
                if any(state in Count and Count[state] == count for state in station_states):
                    heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    return final_stations

# Measure runtime
def measure_runtime(func, *args):
    start_time = timeit.default_timer()
    result = func(*args)
    end_time = timeit.default_timer()
    return result, end_time - start_time

# Run and measure algorithms
greedy_result, greedy_time = measure_runtime(greedy_set_covering, states_needed.copy(), stations.copy())
heap_result, heap_time = measure_runtime(heap_set_covering, states_needed.copy(), stations.copy())
ilp_result, ilp_time = measure_runtime(ilp_set_covering, states_needed.copy(), stations.copy())

# Print results
print(f"Greedy Algorithm: {greedy_result}, Time: {greedy_time}")
print(f"Heap-Based Algorithm: {heap_result}, Time: {heap_time}")
print(f"ILP Algorithm: {ilp_result}, Time: {ilp_time}")

# Summary
print(f"Greedy Algorithm Time: {greedy_time}")
print(f"Heap-Based Algorithm Time: {heap_time}")
print(f"ILP Algorithm Time: {ilp_time}")

Greedy Algorithm: {'E', 'A'}, Time: 1.1156000027767732e-05
Heap-Based Algorithm: {'E', 'A'}, Time: 4.9620000027061906e-05
ILP Algorithm: ['A', 'E'], Time: 0.009903937000217411
Greedy Algorithm Time: 1.1156000027767732e-05
Heap-Based Algorithm Time: 4.9620000027061906e-05
ILP Algorithm Time: 0.009903937000217411
